In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 16
tree_depth = 12
batch_size = 512
device = 'cpu'
train_data_path = r'<>/mitbih_train.csv'  # replace <> with the correct path of the dataset
test_data_path = r'<>/mitbih_test.csv'  # replace <> with the correct path of the dataset

In [3]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [4]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [5]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [6]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [ ]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

/home/eitan.k/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.289229393005371 | KNN Loss: 5.670152187347412 | CLS Loss: 1.6190773248672485
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 4.54009485244751 | KNN Loss: 3.583883047103882 | CLS Loss: 0.9562118053436279
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 4.100872993469238 | KNN Loss: 3.426361560821533 | CLS Loss: 0.6745115518569946
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 3.9538044929504395 | KNN Loss: 3.323531150817871 | CLS Loss: 0.6302732229232788
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 3.8361518383026123 | KNN Loss: 3.2286794185638428 | CLS Loss: 0.6074724793434143
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 3.7472877502441406 | KNN Loss: 3.171922206878662 | CLS Loss: 0.5753654837608337
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 3.6468231678009033 | KNN Loss: 3.116105556488037 | CLS Loss: 0.5307176113128662
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 3.6574208736419678 | KNN Loss: 3.111997127532959 | C

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 3.2328104972839355 | KNN Loss: 3.114835500717163 | CLS Loss: 0.1179749146103859
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 3.244462728500366 | KNN Loss: 3.0576844215393066 | CLS Loss: 0.18677841126918793
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 3.216850757598877 | KNN Loss: 3.074451208114624 | CLS Loss: 0.14239966869354248
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 3.218621253967285 | KNN Loss: 3.038322687149048 | CLS Loss: 0.18029865622520447
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 3.227478265762329 | KNN Loss: 3.080357313156128 | CLS Loss: 0.147120863199234
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 3.2290446758270264 | KNN Loss: 3.0661919116973877 | CLS Loss: 0.16285265982151031
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 3.168565511703491 | KNN Loss: 3.079369068145752 | CLS Loss: 0.08919636905193329
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 3.2440149784088135 | KNN Loss: 3.03478789

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 3.1802217960357666 | KNN Loss: 3.0711419582366943 | CLS Loss: 0.10907985270023346
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 3.1237998008728027 | KNN Loss: 3.055849552154541 | CLS Loss: 0.06795023381710052
Epoch: 007, Loss: 3.1563, Train: 0.9763, Valid: 0.9734, Best: 0.9734
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 3.1230552196502686 | KNN Loss: 3.047787666320801 | CLS Loss: 0.07526752352714539
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 3.1652050018310547 | KNN Loss: 3.0534515380859375 | CLS Loss: 0.11175335198640823
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 3.1704797744750977 | KNN Loss: 3.0480620861053467 | CLS Loss: 0.12241765111684799
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 3.116887331008911 | KNN Loss: 3.026285409927368 | CLS Loss: 0.09060196578502655
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 3.147453546524048 | KNN Loss: 3.0419952869415283 | CLS Loss: 0.10545828193426132
Epoch 8 / 200 | it

Epoch 11 / 200 | iteration 50 / 171 | Total Loss: 3.1250813007354736 | KNN Loss: 3.0158817768096924 | CLS Loss: 0.10919956862926483
Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 3.102080821990967 | KNN Loss: 3.026804208755493 | CLS Loss: 0.07527655363082886
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 3.106606960296631 | KNN Loss: 2.999403715133667 | CLS Loss: 0.10720334947109222
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 3.098684310913086 | KNN Loss: 3.0441627502441406 | CLS Loss: 0.05452164262533188
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 3.1193268299102783 | KNN Loss: 3.0435986518859863 | CLS Loss: 0.07572827488183975
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 3.141392469406128 | KNN Loss: 3.0385537147521973 | CLS Loss: 0.10283870249986649
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 3.1121749877929688 | KNN Loss: 3.045490026473999 | CLS Loss: 0.06668485701084137
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 3.1047325134277344 | KNN Loss:

Epoch 14 / 200 | iteration 120 / 171 | Total Loss: 3.1225779056549072 | KNN Loss: 3.07989239692688 | CLS Loss: 0.042685553431510925
Epoch 14 / 200 | iteration 130 / 171 | Total Loss: 3.1413917541503906 | KNN Loss: 3.039933443069458 | CLS Loss: 0.10145843029022217
Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 3.1185741424560547 | KNN Loss: 3.0359280109405518 | CLS Loss: 0.08264607936143875
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 3.107323408126831 | KNN Loss: 3.051271677017212 | CLS Loss: 0.0560518354177475
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 3.114885091781616 | KNN Loss: 3.051020383834839 | CLS Loss: 0.06386478990316391
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 3.0773465633392334 | KNN Loss: 3.0175700187683105 | CLS Loss: 0.05977657064795494
Epoch: 014, Loss: 3.1123, Train: 0.9830, Valid: 0.9798, Best: 0.9799
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 3.1257026195526123 | KNN Loss: 3.0633950233459473 | CLS Loss: 0.06230752915143967
Epoch 15 

Epoch 18 / 200 | iteration 10 / 171 | Total Loss: 3.047639846801758 | KNN Loss: 3.0063743591308594 | CLS Loss: 0.041265495121479034
Epoch 18 / 200 | iteration 20 / 171 | Total Loss: 3.1023342609405518 | KNN Loss: 3.0273938179016113 | CLS Loss: 0.07494048029184341
Epoch 18 / 200 | iteration 30 / 171 | Total Loss: 3.078040361404419 | KNN Loss: 3.0472617149353027 | CLS Loss: 0.030778763815760612
Epoch 18 / 200 | iteration 40 / 171 | Total Loss: 3.0831592082977295 | KNN Loss: 3.0112509727478027 | CLS Loss: 0.07190817594528198
Epoch 18 / 200 | iteration 50 / 171 | Total Loss: 3.077197790145874 | KNN Loss: 3.03450870513916 | CLS Loss: 0.042689040303230286
Epoch 18 / 200 | iteration 60 / 171 | Total Loss: 3.102269411087036 | KNN Loss: 3.064713954925537 | CLS Loss: 0.03755553066730499
Epoch 18 / 200 | iteration 70 / 171 | Total Loss: 3.0684375762939453 | KNN Loss: 3.0333611965179443 | CLS Loss: 0.03507634997367859
Epoch 18 / 200 | iteration 80 / 171 | Total Loss: 3.091987133026123 | KNN Loss: 

Epoch 21 / 200 | iteration 80 / 171 | Total Loss: 3.064152956008911 | KNN Loss: 3.033048391342163 | CLS Loss: 0.031104493886232376
Epoch 21 / 200 | iteration 90 / 171 | Total Loss: 3.0890579223632812 | KNN Loss: 3.062134027481079 | CLS Loss: 0.026923948898911476
Epoch 21 / 200 | iteration 100 / 171 | Total Loss: 3.102947950363159 | KNN Loss: 3.0143537521362305 | CLS Loss: 0.08859430253505707
Epoch 21 / 200 | iteration 110 / 171 | Total Loss: 3.084491491317749 | KNN Loss: 3.0092668533325195 | CLS Loss: 0.07522459328174591
Epoch 21 / 200 | iteration 120 / 171 | Total Loss: 3.1031434535980225 | KNN Loss: 3.0316169261932373 | CLS Loss: 0.0715264230966568
Epoch 21 / 200 | iteration 130 / 171 | Total Loss: 3.0821657180786133 | KNN Loss: 3.0285468101501465 | CLS Loss: 0.05361894890666008
Epoch 21 / 200 | iteration 140 / 171 | Total Loss: 3.102811813354492 | KNN Loss: 3.0358939170837402 | CLS Loss: 0.06691781431436539
Epoch 21 / 200 | iteration 150 / 171 | Total Loss: 3.092658281326294 | KNN L

In [ ]:
test(model, test_data_iter, device)

In [ ]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

In [ ]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

In [ ]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [ ]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

In [ ]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [ ]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [ ]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [ ]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [ ]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [ ]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [ ]:
losses = []
accs = []
sparsity = []

In [ ]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

In [ ]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

# Tree Visualization

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

# Extract Rules

# Accumulate samples in the leaves

In [ ]:
print(f"Number of patterns: {len(root.get_leaves())}")

In [ ]:
method = 'greedy'

In [ ]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [ ]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")